# IoT Equipment Failure Prediction using Sensor data
## 1 Environment Setup
### 1.1 Import dependent libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import pdb
import json
import re
import requests

In [2]:
# Import libraries
from io import StringIO
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

## 2 Create IoT Predictive Analytics Functions

In [3]:
# Function to extract Column names of dataset
def dataset_columns(dataset):
    return list(dataset.columns.values)

# Function to train Logistic regression model
def train_logistic_regression(x_vals, y_vals):
    logistic_regression_model = LogisticRegression()
    logistic_regression_model.fit(x_vals, y_vals)
    return logistic_regression_model

# Function to return Predicted values
def score_data(trained_model, x_vals):
    ypredict = trained_model.predict(x_vals)
    return ypredict

# Function to calculate Prediction accuracy of model
def model_accuracy(trained_model, variables, targets):
    accuracy_score = trained_model.score(variables, targets)
    return accuracy_score

# Function to generate Confusion matrix
def confusion_matrix(actfail, predictfail):
  # Compute Confusion matrix
  print("Actual, Predicted Observations:  ",len(actfail), len(predictfail))
  # print(actfail, predictfail)
  anpn = 0
  anpy = 0
  aypn = 0
  aypy = 0
  
  for i in range(len(actfail)):
      if (actfail[i]==0 and predictfail[i]==0):
          anpn = anpn + 1
      elif (actfail[i]==0 and predictfail[i]==1):
          anpy = anpy + 1
      elif (actfail[i]==1 and predictfail[i]==0):
          aypn = aypn + 1
      else:
          aypy = aypy + 1
  # Confusoin matrix
  print "Confusion Matrix"
  print "--------------------------------------------"
  print "              ", "Predicted N", "Predicted Y"
  print "Actual N      ", anpn,"          ", anpy 
  print "Actual Y      ", aypn,"          ", aypy
  print "--------------------------------------------"
  print "Total observations:  ", anpn+anpy+aypn+aypy
  print "False Positives   :  ", anpy
  print "False Negatives   :  ", aypn
  print "Overall Accuracy  :  ", (float(anpn+aypy)/float(anpn+anpy+aypn+aypy))*100
  print "--------------------------------------------"


## 3 Configure Parameters for Change Point Detection
### 3.1 Read DSX Configuration file and load all parameters

Complete below 2 steps before executing the rest of the cells

1. Configure the parameters in JSON file and upload to Object storage
2. Set the Configuration .json file name in the next section


#### 3.1.1 Set the name of the .json configuration file

In [4]:
# Specify file names for configuration files
v_sampleConfigFileName = "iotpredict_config.json"

#### 3.1.2 Insert the Object Storage file credentials to read the .json configuration file

In [5]:
# @hidden_cell
# The section below needs to be modified:
#    Insert your credentials to read data from your data sources and replace 
#    the idaConnect() section below
# This function accesses a file in your Object Storage. The definition contains your credentials.
def get_object_storage_file_with_credentials_273b1c76068e4fe4b6cb7633e12004f3(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_72d4a7a9b40e88f4d982e9f60da535024f8e02ad','domain': {'id': '1301cc61df814635b2dd7c9fa40e6e2a'},
            'password': 'Weu0?0A7hJU~#b?j'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)


### 3.2 Read Configuration parametric values

In [6]:
# Function to Read json parametric values
def f_getconfigval(injsonstr, invarname):
    # paramname, paramvalue
    # Unpack the json parameter values
    # This section requires regex
    for i in range(len(injsonstr)):
        pair = injsonstr[i]
        # Return parametric value
        if pair['paramname'] == invarname:
            return(pair['paramvalue'])

In [7]:
# Read configuration parameters from JSON file
# @hidden_cell
# The section below needs to be modified:
#    Insert your credentials to read data from your data sources and replace 
#    the idaConnect() section below
# This function accesses a file in your Object Storage. The definition contains your
#    credentials
# Your data file was loaded into a StringIO object and you can process the data.
# Please read the documentation of pandas to learn more about your possibilities to load your data.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html
inputfo = get_object_storage_file_with_credentials_273b1c76068e4fe4b6cb7633e12004f3('IoTPredictive',
                                                                                    v_sampleConfigFileName)

d = json.load(inputfo)


In [8]:
# Read JSON configuration parametric values
# Unpack the json parameter values
# This section uses regex
for i in range(len(d)):
  pair = d[i]
  # print pair
  if pair['paramname'] == "features":
    text = pair['paramvalue']
    feature_list = eval("list("+ text +")")
  if pair['paramname'] == "target":
    target = str(pair['paramvalue'])
  if pair['paramname'] == "data_size":
    train_datasize = float(pair['paramvalue'])


v_feature_list = eval("list("+ f_getconfigval(d, "features") +")")
v_target = str(f_getconfigval(d, "target"))
v_train_datasize = float(f_getconfigval(d, "data_size"))


In [9]:
# Verify configuration parametric values
# print (feature_list, target, train_datasize)
print (v_feature_list, v_target, v_train_datasize)

(['atemp', 'PID', 'outpressure', 'inpressure', 'temp'], 'fail', 0.7)


## 4 Read IoT Sensor data from database

In [10]:
# Read data from DB2 warehouse database
from ibmdbpy import IdaDataBase, IdaDataFrame

# Call function to read data for specific sensor
# @hidden_cell
# The section below needs to be modified:
#    Insert your credentials to read data from your data sources and replace 
#    the idaConnect() section below
# This connection object is used to access your data and contains your credentials.
idadb_d281f6cd34eb4bc98f0183a45598dbb9 = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-entry-yp-lon02-01.services.eu-gb.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=dash100002;PWD=FNw_7glWu9B_')
df_iotdata = IdaDataFrame(idadb_d281f6cd34eb4bc98f0183a45598dbb9, 'DASH100002.IOT_SENSOR_DATA').as_dataframe()

# Check Number of observations read for analysis
print "Number of Observations :", len(df_iotdata)
# Inspect a few observations
df_iotdata.head()


Number of Observations : 944


,popul,atemp,selfLR,ClinLR,DoleLR,PID,outpressure,inpressure,temp,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0


In [11]:
# Print dataset column names
datacolumns = dataset_columns(df_iotdata)
print "Data set columns : ", list(datacolumns)

Data set columns :  ['popul', 'atemp', 'selfLR', 'ClinLR', 'DoleLR', 'PID', 'outpressure', 'inpressure', 'temp', 'fail']


## 5 Run Failure Prediction algorithm on IoT data
### 5.1 Split data into Training and Test data

In [12]:
# Split Training and Testing data
train_x, test_x, train_y, test_y = train_test_split(df_iotdata[v_feature_list], df_iotdata[v_target], train_size=v_train_datasize)
print "Train x counts : ", len(train_x), len(train_x.columns.values)
print "Train y counts : ", len(train_y)
 
print "Test x counts : ", len(test_x), len(test_x.columns.values)
print "Test y counts : ", len(test_y)


Train x counts :  660 5
Train y counts :  660
Test x counts :  284 5
Test y counts :  284


### 5.2 Train the Predictive model

In [13]:
# Training Logistic regression model
trained_logistic_regression_model = train_logistic_regression(train_x, train_y)

train_accuracy = model_accuracy(trained_logistic_regression_model, train_x, train_y)

# Testing the logistic regression model
test_accuracy = model_accuracy(trained_logistic_regression_model, test_x, test_y)

print "Training Accuracy : ", train_accuracy * 100, "%"



Training Accuracy :  91.2121212121 %


### 5.3 Score the Test data using the Trained model

In [14]:
# Model accuracy: Score and construct Confusion matrix for Test data
actfail = test_y.values
predictfail = score_data(trained_logistic_regression_model, test_x)


## 6 Confusion matrix for deeper analysis of Prediction accuracy
#####   Confusion matrix outputs below can be used for calculating more customised Accuracy metrics

In [15]:
# Print Count of Actual fails, Predicted fails
# Print Confusion matrix
confusion_matrix(actfail, predictfail)

('Actual, Predicted Observations:  ', 284, 284)
Confusion Matrix
--------------------------------------------
               Predicted N Predicted Y
Actual N       139            21
Actual Y       11            113
--------------------------------------------
Total observations:   284
False Positives   :   21
False Negatives   :   11
Overall Accuracy  :   88.7323943662
--------------------------------------------
